# Visualizing Augmentations
In this notebook, I am trying to visualize various augmentation techniques available on Albumentations. Albumentations is a Python library for image augmentation. Image augmentation is used in deep learning and computer vision tasks to increase the quality of trained models. The purpose of image augmentation is to create new training samples from the existing data.

Type of augmentations:
1. Pixel-level transforms: Pixel-level transforms will change just an input image and will leave any additional targets such as masks, bounding boxes, and keypoints unchanged. 
2. Spatial-level transforms: Spatial-level transforms will simultaneously change both an input image as well as additional targets such as masks, bounding boxes, and keypoints. 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
import albumentations as A
import random
import torch

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)
    
def visualize_multiple(nrows, ncols, img, transform):
    fig, axes = plt.subplots(nrows,ncols)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    num_iter = 0
    for row in range(nrows):
        for col in range(ncols):
            augmented_img = transform[num_iter](image=img)['image']
            axes[row,col].imshow(augmented_img)
            axes[row,col].grid(False)
            axes[row,col].set_xticks([])
            axes[row,col].set_yticks([])
            num_iter += 1
    return fig, axes

### Sample Image
I pick one image from the Cassava dataset for as a sample.

In [ ]:
seed_everything(100)
img = cv2.imread('../input/cassava-leaf-disease-classification/train_images/100042118.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
visualize(img)

# 1.0. Pixel-Level Transforms
Pixel-level transforms will change just an input image and will leave any additional targets such as masks, bounding boxes, and keypoints unchanged.

## 1.1. Blur
Blur the input image using a random-sized kernel.

In [ ]:
blur_limits = np.arange(3,39,4)
transform = [A.Blur(p=1, blur_limit=[limit,limit], always_apply=True) for limit in blur_limits]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Blur kernel size: ({}, {})'.format(blur_limits[num_iter], blur_limits[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.2. CLAHE
Apply Contrast Limited Adaptive Histogram Equalization to the input image.

In [ ]:
params = np.arange(3,30,3)
transform = [A.CLAHE(clip_limit=[param, param], tile_grid_size=(param, param), always_apply=True) for param in params]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Clip limit: ({}, {}), tile grid size: ({}, {})'.format(params[num_iter], params[num_iter], params[num_iter], params[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.3. Channel Dropout
Randomly Drop Channels in the input Image.

In [ ]:
transform = [A.ChannelDropout(channel_drop_range=(1,2), always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.4. Channel Shuffle
Randomly rearrange channels of the input RGB image.

In [ ]:
transform = [A.ChannelShuffle(p=1) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.5. Color Jitter
Randomly changes the brightness, contrast, and saturation of an image. Compared to ColorJitter from torchvision, this transform gives a little bit different results because Pillow (used in torchvision) and OpenCV (used in Albumentations) transform an image to HSV format by different formulas.

In [ ]:
transform = [A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.6. Downscale
Decreases image quality by downscaling and upscaling back.

In [ ]:
transform = [A.Downscale(scale_min=0.25, scale_max=0.25, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.7. Equalize
Equalize the image histogram.

In [ ]:
params = [['cv', True], ['cv', False], ['pil', True], ['pil', False]]
transform = [A.Equalize(mode=param[0], by_channels=param[1], always_apply=True) for param in params]
fig, axes = visualize_multiple(2,2,img,transform)

num_iter = 0
for row in range(2):
    for col in range(2):
        text = 'Mode: {}, By channels: {}'.format(params[num_iter][0], params[num_iter][1])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.8. Fancy PCA
Augment RGB image using FancyPCA from Krizhevsky's paper "ImageNet Classification with Deep Convolutional Neural Networks"

In [ ]:
params = np.arange(0.1,1.0,0.1)
transform = [A.FancyPCA(alpha=param, always_apply=True) for param in params]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Alpha: {:.1f}'.format(params[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.9. Gaussian Noise
Apply gaussian noise to the input image.

In [ ]:
params = np.arange(1000, 10000, 1000)
transform = [A.GaussNoise(var_limit=(param-500, param), mean=0, always_apply=True) for param in params]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Variance range: ({}, {})'.format(params[num_iter]-500, params[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.10. Gaussian Blur
Blur the input image using a Gaussian filter with a random kernel size.

In [ ]:
blur_limits = np.arange(3,39,4)
transform = [A.GaussianBlur(blur_limit=(limit, limit), always_apply=True) for limit in blur_limits]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Kernel size: ({}, {})'.format(blur_limits[num_iter], blur_limits[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.11. Glass Blur
Apply glass noise to the input image.

In [ ]:
sigmas = np.arange(0.5, 5.0, 0.5)
transform = [A.GlassBlur(sigma=sigma, max_delta=4, iterations=2, always_apply=True) for sigma in sigmas]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Sigma: {}'.format(sigmas[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.12. Hue Saturation Value
Randomly change hue, saturation and value of the input image.

In [ ]:
transform = [A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=True) for sigma in sigmas]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.13. IAA Additive Gaussian Noise
Add gaussian noise to the input image.

In [ ]:
locs = np.arange(10,100,10)
transform = [A.IAAAdditiveGaussianNoise(loc=loc, scale=(2.5500000000000003, 12.75), always_apply=True) for loc in locs]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Mean: {}'.format(locs[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.14. IAA Sharpen
Sharpen the input image and overlays the result with the original image.

In [ ]:
alphas = np.arange(0.1, 1.0, 0.1)
lightness = np.arange(0.1, 1.0, 0.1)
transform = [A.IAASharpen(alpha=(alpha, alpha), lightness=(light, light), always_apply=True) for alpha, light in zip(alphas, lightness)]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Alpha: {:.1f}, Lightness: {:.1f}'.format(alphas[num_iter], lightness[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.15. IAA Superpixels
Completely or partially transform the input image to its superpixel representation. Uses skimage's version of the SLIC algorithm. May be slow.

In [ ]:
transform = [A.IAASuperpixels(p_replace=0.1, n_segments=100, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.16. ISO Noise
Apply camera sensor noise.

In [ ]:
color_shifts = np.arange(0.01, 0.1, 0.01)
intensities = np.arange(0.1, 1.0, 0.1)
transform = [A.ISONoise(color_shift=(shift, shift), intensity=(intensity, intensity), always_apply=True) for shift, intensity in zip(color_shifts, intensities)]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Color shift: {:.2f}, Intensity: {:.2f}'.format(color_shifts[num_iter], intensities[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.17. Invert Image
Invert the input image by subtracting pixel values from 255.

In [ ]:
transform = [A.InvertImg(p=1.0), A.InvertImg(p=0.0), A.InvertImg(p=0.0), A.InvertImg(p=1.0)]
fig, axes = visualize_multiple(2,2,img,transform)

num_iter = 0
for row in range(2):
    for col in range(2):
        text = 'Inverted' if num_iter==0 or num_iter==3 else 'Normal'
        axes[row, col].set_title(text)
        num_iter += 1

## 1.18. Median Blur
Blur the input image using a median filter with a random aperture linear size.

In [ ]:
blur_limits = np.arange(5, 30, 2)
transform = [A.MedianBlur(blur_limit=(limit,limit), always_apply=True) for limit in blur_limits]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Blur limit: {}'.format(blur_limits[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.19. Motion Blur
Apply motion blur to the input image using a random-sized kernel.

In [ ]:
blur_limits = np.arange(7, 30, 2)
transform = [A.MotionBlur(blur_limit=(limit,limit), always_apply=True) for limit in blur_limits]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Blur limit: {}'.format(blur_limits[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.20. Multiplicative Noise
Multiply image to random number or array of numbers.

In [ ]:
multipliers = np.arange(0.5, 1.5, 0.1)
transform = [A.MultiplicativeNoise(multiplier=plier, always_apply=True) for plier in multipliers]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Multiplier: {:.1f}'.format(multipliers[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.21. Posterize
Reduce the number of bits for each color channel.

In [ ]:
list_num_bits = np.arange(0,9,1)
transform = [A.Posterize(num_bits=int(num_bits), always_apply=True) for num_bits in list_num_bits]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Num bits: {}'.format(list_num_bits[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.22. RGB Shift
Randomly shift values for each channel of the input RGB image.

In [ ]:
transform = [A.RGBShift(always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.23. Random Brightness Contrast
Randomly change brightness and contrast of the input image.

In [ ]:
transform = [A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.24. Random Fog
Simulates fog for the image

In [ ]:
transform = [A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=1, alpha_coef=0.08, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.25. Random Gamma

In [ ]:
transform = [A.RandomGamma(gamma_limit=(50, 250), always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.26. Random Rain
Adds rain effects.

In [ ]:
rain_types = [None, 'drizzle', 'heavy', 'torrential']
transform = [A.RandomRain(slant_lower=-10, slant_upper=10, 
                          drop_length=20, drop_width=1, drop_color=(200, 200, 200), 
                          blur_value=7, brightness_coefficient=0.7, 
                          rain_type=rain_type, always_apply=True) for rain_type in rain_types]
fig, axes = visualize_multiple(2,2,img,transform)

num_iter = 0
for row in range(2):
    for col in range(2):
        text = 'Rain type: {}'.format(rain_types[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 1.27. Random Shadow
Simulates shadows for the image

In [ ]:
transform = [A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), 
                            num_shadows_lower=1, num_shadows_upper=4, 
                            shadow_dimension=5, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.28. Random Snow 
Bleach out some pixel values simulating snow.

In [ ]:
transform = [A.RandomSnow(snow_point_lower=0.1, 
                          snow_point_upper=0.3, 
                          brightness_coeff=2.5, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.29. Random Sun Flare
Simulates Sun Flare for the image

In [ ]:
transform = [A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), 
                              angle_lower=0, angle_upper=1, 
                              num_flare_circles_lower=6, 
                              num_flare_circles_upper=10, 
                              src_radius=400, src_color=(255, 255, 255),
                              always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 1.30. Solarize
Invert all pixel values above a threshold.

In [ ]:
thresholds = np.arange(10, 255, 20)
transform = [A.Solarize(threshold=int(thresh), always_apply=True) for thresh in thresholds]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Threshold: {}'.format(thresholds[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

# 2.0. Spatial-Level Transform
Spatial-level transforms will simultaneously change both an input image as well as additional targets such as masks, bounding boxes, and keypoints.

## 2.1. Center Crop
Crop the central part of the input.

In [ ]:
crop_sizes = np.arange(50, 500, 50)
transform = [A.CenterCrop(height=crop_size, width=crop_size, always_apply=True) for crop_size in crop_sizes]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Crop size: ({}, {})'.format(crop_sizes[num_iter], crop_sizes[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 2.2. Coarse Dropout
CoarseDropout of the rectangular regions in the image.

In [ ]:
hole_sizes = np.arange(10, 100, 10)
transform = [A.CoarseDropout(max_holes=8, max_height=hole_size, max_width=hole_size, 
                             min_holes=None, min_height=hole_size, min_width=hole_size,
                             always_apply=True) for hole_size in hole_sizes]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Hole size: ({}, {})'.format(hole_sizes[num_iter], hole_sizes[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 2.3. Crop
Crop region from image.**

In [ ]:
transform = [A.Crop(x_min=random.randint(0, 100), 
                    y_min=random.randint(0, 75), 
                    x_max=random.randint(200, img.shape[1]), 
                    y_max=random.randint(200, img.shape[0]), 
                    always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.4. Flip
Flip the input either horizontally, vertically or both horizontally and vertically.

In [ ]:
transform = [A.Flip(p=0.75) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.5. Grid Distortion

In [ ]:
transform = [A.GridDistortion(always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.6. Grid Dropout
GridDropout, drops out rectangular regions of an image and the corresponding mask in a grid fashion.

In [ ]:
transform = [A.GridDropout(ratio=0.5, 
                           unit_size_min=random.randint(0, 50), 
                           unit_size_max=random.randint(60, 100), 
                           holes_number_x=random.randint(0, 5), 
                           holes_number_y=random.randint(0, 5), 
                           always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.7. IAA Affine
Place a regular grid of points on the input and randomly move the neighbourhood of these point around via affine transformations.

In [ ]:
transform = [A.IAAAffine(scale=1.0, 
                         translate_percent=random.randint(0,20), 
                         translate_px=None, 
                         rotate=random.randint(0,360), 
                         shear=random.randint(0,10), 
                         order=1, 
                         cval=0, 
                         mode='reflect',
                         always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.8. Optical Distortion

In [ ]:
transform = [A.OpticalDistortion(distort_limit=0.75, shift_limit=0.75, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.9. Random Grid Shuffle
Random shuffle grid's cells on image.

In [ ]:
grid_sizes = np.arange(2, 11, 1)
transform = [A.RandomGridShuffle(grid=(size,size), always_apply=True) for size in grid_sizes]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Grid size: ({}, {})'.format(grid_sizes[num_iter], grid_sizes[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

## 2.10. Random Resized Crop
Torchvision's variant of crop a random part of the input and rescale it to some size.

In [ ]:
transform = [A.RandomResizedCrop(height=300, width=400, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.11. Random Rotate 90
Randomly rotate the input by 90 degrees zero or more times.

In [ ]:
transform = [A.RandomRotate90(p=0.75) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.12. Random Scale
Randomly resize the input. Output image size is different from the input image size.

In [ ]:
transform = [A.RandomScale(scale_limit=0.5, always_apply=True) for _ in range(9)]
fig, axes = visualize_multiple(3,3,img,transform)

## 2.13. Transpose
Transpose the input by swapping rows and columns.

In [ ]:
transform = [A.Transpose(p=0.5) for _ in range(9)]
fig, axes = visualize_multiple(2,2,img,transform)